In [38]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [39]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WATCH_DIRECTORY = os.getenv("WATCH_DIRECTORY")
OPENAI_ENGINE = os.getenv("OPENAI_ENGINE")
GRAPHD_HOST = os.getenv("GRAPHD_HOST")
GRAPHD_PORT = os.getenv("GRAPHD_PORT")
NEBULA_USER = os.getenv("NEBULA_USER")
NEBULA_PASSWORD = os.getenv("NEBULA_PASSWORD")
NEBULA_ADDRESS = os.getenv("NEBULA_ADDRESS")

In [40]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)

from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    ServiceContext,
)

from llama_index import set_global_service_context

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

from IPython.display import Markdown, display

from llama_index.llms import OpenAI

from pathlib import Path
from llama_index import download_loader



# define LLM
llm = OpenAI(temperature=0, model=OPENAI_ENGINE)
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

# Define Service Context

In [41]:
# set global service context
set_global_service_context(service_context)

# Install WSL (Windows Sub-system for Linux) for running Nebula Graph DB Locally with Docker Desktop

1. Move to Home Directory:

>cd ~

2. Create a new Directory:

>mkdir ~/nebula-up-dir

>cd ~/nebula-up-dir

3. Clone the following repository for bootstrap server:

https://github.com/wey-gu/nebula-up.git

4. Run the installation command:

>cd nebula-up

>curl -fsSL nebula-up.siwei.io/install.sh | bash

# Docker Desktop Approach

List Docker Network : docker network ls
Add Hosts for Stirage Service
1. Access Nebula Graph Console:

>docker network ls

>docker network inspect weygu_nebulagraph-dd-ext-desktop-extension_nebula-net (check if 9 containers are present in the network)

2. Connect to Nebula Graph Console:

>docker run --rm -ti --network=weygu_nebulagraph-dd-ext-desktop-extension_nebula-net vesoft/nebula-console:v3 -u <USER_NAME> -p <PASSWORD> --address=nebulagraph_graphd --port=9669 

3. Open another CMD

>docker ps

>docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' nebulagraph_storaged0

>docker inspect -f '{{range.NetworkSettings.Networks}}{{.IPAddress}}{{end}}' nebulagraph_storaged1

4. Add Storage Host in Nebula Graph Console:

>(root@nebula) [(none)]> ADD HOSTS '172.18.0.11':9779, '172.18.0.9':9779, '172.18.0.10':9779;

>(root@nebula) [(none)]> SHOW HOSTS;

5. Change local_ip in nebulagraph_storaged0, nebulagraph_storaged1, nebulagraph_storaged2 configuration files:

>docker exec -it nebulagraph_storaged0 bash

>cd etc

>ls
nebula-storaged-listener.conf.production  nebula-storaged.conf  nebula-storaged.conf.default  nebula-storaged.conf.production

>vi nebula-storaged.conf

Change local_ip=172.18.0.11

Change local_ip=172.18.0.9

Change local_ip=172.18.0.10



In [42]:
%load_ext ngql
connection_string = f"--address {os.environ['GRAPHD_HOST']} --port 9669 --user root --password {os.environ['NEBULA_PASSWORD']}"
%ngql {connection_string}
%ngql USE demo_basketballplayer

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created
INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)
INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)
[ERROR]:
 Query Failed:
 SpaceNotFound: SpaceName `demo_basketballplayer`


""


In [43]:
%ngql SHOW SPACES;

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


,Name
0,synthia_knowledge_graph


In [44]:
%ngql CREATE SPACE IF NOT EXISTS synthia_knowledge_graph(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


""


In [45]:
%ngql SHOW SPACES;

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


,Name
0,synthia_knowledge_graph


In [46]:
%%ngql 
USE synthia_knowledge_graph;
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


""


In [47]:
%ngql CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


""


In [48]:
#%ngql USE rag_workshop; CLEAR SPACE rag_workshop; # clean graph space

# Storage_Context with Graph_Store

In [49]:
os.environ['NEBULA_USER'] = os.environ["NEBULA_USER"]
os.environ['NEBULA_PASSWORD'] = os.environ["NEBULA_PASSWORD"]
os.environ['NEBULA_ADDRESS'] = os.environ["NEBULA_ADDRESS"]

space_name = "synthia_knowledge_graph"
edge_types, rel_prop_names = ["relationship"], ["relationship"]
tags = ["entity"]

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# Knowledge Graph building with Llama Index

In [50]:
# Retrieve the Documents - pages loaded with Langchain in data_preparation.ipynb for using with Llama Index to extract triplets and save to NebulaGraph
import pickle
with open('documents.pkl', 'rb') as file:
    documents = pickle.load(file)

In [51]:
documents

[Document(page_content='Office of Employee Relations  \nExempt Officers’ and Sergeants’ Modified Duty Program  \n                                                                                           \n   Original Effective Date: June 21, 2005  \nRevised Effective Date:  May 11, 2020  \nPage 1 of 6 \n BACKGROUND  \n1. The City and San Jose Police Officers ’ Association (SJPOA) recognize that, despite best \nefforts to promote safety, police officers and sergeants are injured in the line of duty. Such \ninjuries are unfortunate but can be a consequence of police work. The Exempt Officers ’ \nand Sergeants ’ Modified Duty Program (“Program ”) is available to any police officer or \nsergeant that has work -related or non -work related injuries or illnesses which preclude \nhim or her from performing the f ull scope of his or her duties without accommodation.  \n2. The City and SJPOA recognize that police officers and sergeants exist to enforce the law \nand protect public safety. Some

In [52]:
def get_pdf_paths(directory):
    """
    This function scans the specified directory and returns the file paths of all PDF files in it.
    """
    pdf_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_paths.append(os.path.join(root, file))
    return pdf_paths

# Usage
directory = WATCH_DIRECTORY
pdf_paths = get_pdf_paths(directory)
print(pdf_paths)

['C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Article 39 - Exempt Officers and Sergeants Modified Duty Program.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Association of Building Mechanical and Electrical Inspectors (ABMEI) MOA.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Association of Engineers and Architects IFPTE Local 21 Unit 43 MOA.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Association of Engineers and Architects IFTPE Local 21 Units 4142 MOA.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Association of Legal Professionals of San Jose (ALP).pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Association of Maintenance Supervisory Personnel IFPTE Local 21 (AMSP) MOA.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\City Association of Management Personnel IFPTE Local 21 (CAMP) MOA.pdf', 'C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\

In [53]:
def fetch_pages(pdf_paths):
    for file_path in pdf_paths:
        UnstructuredReader  = download_loader("UnstructuredReader", refresh_cache=True)
        loader = PDFReader()
        documents = loader.load_data(WATCH_DIRECTORY, split_documents = False)

In [54]:
documents[100]

Document(page_content='AEA (Units 41 & 42) MOA  July 1, 2021 – June 30, 2023     Page 8 6.2.7  The parties agree that they have a mu tual i nterest in well -trained Representatives.  \nToward this end, up to four (4) designated  Representative s shall be granted a \nmaximum of eight (8) hours paid release time during each year of this agreement \nto participate in training sessions related to the  provi sions of this agreeme nt, \njointly conducted by the Union and the Office of Employee R elations, accor ding \nto an outline of such training activities to be submitted by the Union and approved \nby the Office of Employee Relations prior to conducting any su ch tra ining \nsessions.  \n \n6.3 Release Time  \n \n6.3.1  Release time from regular City duties shall  be provided to  designated Union \nrepresentatives in accordance with the following provisions.  \n \n6.3.2  Designated Union Representatives .  The following designated Union  \nRepre sentatives shall be e ligible for release 

In [55]:
!pip install PyPDF2

In [56]:
!{sys.executable} -m pip install PyPDF2

In [57]:
# Import necessary libraries
from pydantic import BaseModel, Field
from pathlib import Path
from typing import IO, Dict, List, Optional, Union, Any
import uuid
from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document
from PyPDF2 import PdfReader

# Define the ExtendedDocument class
class ExtendedDocument(Document):
    id_: Optional[str] = Field(None, alias='id')
    embedding: Optional[Any] = None
    hash: Optional[str] = None

# Define a function to generate unique IDs
def generate_id():
    return str(uuid.uuid4())

# Define a custom PDFReader class
class PDFReader(BaseReader):
    def load_data(self, file: Union[IO[bytes], str, Path], extra_info: Optional[Dict] = None) -> List[ExtendedDocument]:
        if not isinstance(file, Path) and isinstance(file, str):
            file = Path(file)

        context = open(file, "rb") if isinstance(file, Path) else file

        with context as fp:
            pdf = PdfReader(fp)
            num_pages = len(pdf.pages)

            docs = []
            for page in range(num_pages):
                page_text = pdf.pages[page].extract_text()
                metadata = {"page_number": page, "file_name": file.name}

                if extra_info is not None:
                    metadata.update(extra_info)

                # Create an ExtendedDocument instance
                docs.append(ExtendedDocument(text=page_text, extra_info=metadata, id_=generate_id()))
            return docs

# Example usage
pdf_reader = PDFReader()
sample_extended_documents = pdf_reader.load_data('C:/Users/koush/Synthia_Anaconda/src/synthia/notebooks/data\\Article 39 - Exempt Officers and Sergeants Modified Duty Program.pdf')

In [58]:
sample_extended_documents

[ExtendedDocument(id_='79f16646-4af1-4590-a5d5-69dcf6979ebd', embedding=None, metadata={'page_number': 0, 'file_name': 'Article 39 - Exempt Officers and Sergeants Modified Duty Program.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='6f4c577595eda38d7d4f8bc70d82cc84c73d38ae3456f16a38fa90a8dac8c5e8', text='Office of Employee Relations  \nExempt Officers’ and Sergeants’ Modified Duty Program  \n                                                                                           \n   Original Effective Date: June 21, 2005  \nRevised Effective Date:  May 11, 2020  \nPage 1 of 6 \n BACKGROUND  \n1. The City and San Jose Police Officers ’ Association (SJPOA) recognize that, despite best \nefforts to promote safety, police officers and sergeants are injured in the line of duty. Such \ninjuries are unfortunate but can be a consequence of police work. The Exempt Officers ’ \nand Sergeants ’ Modified Duty Program (“Program ”) is available to an

In [37]:
kg_index_extended_docs = KnowledgeGraphIndex.from_documents(
    sample_extended_documents,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

In [61]:
from llama_index import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents_wiki = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)

In [62]:
documents_wiki[0]

Document(id_='e27cb9d9-7dff-4cff-8386-2a7ea2c4056d', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='64f4a75e473f43f46c0c299d3dd5090dac4863fbece55acb3d09c5c9e56ced8d', text='Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios, and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the Guardians must protect Rocket (Cooper)

In [63]:
kg_index_wiki = KnowledgeGraphIndex.from_documents(
    documents_wiki,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

# Persist Storage Context

In [69]:
kg_index_wiki.storage_context.persist(persist_dir='./data/storage_graph/')

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [70]:
import os

# Specify the directory you want to list
directory = './data/storage_graph'

# List files and directories in the specified directory
files = os.listdir(directory)
for file in files:
    print(file)

docstore.json
index_store.json
vector_store.json


# Restore storage_context from disk

In [71]:
from llama_index import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./data/storage_graph/', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.indices.loading:Loading all indices.


# Approach 1 : Text2Cypher

In [72]:
from llama_index.query_engine import KnowledgeGraphQueryEngine

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore

nl2kg_query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
)

In [73]:
# activate connections
%ngql SHOW HOSTS
r = nl2kg_query_engine.query("SHOW HOSTS")

INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)


In [74]:
question = """
Why does the San Jose Police Department have an interest in maximizing the number of officers and sergeants available for Patrol duties?
"""

response_nl2kg = nl2kg_query_engine.query(question)

# Cypher:

print("The Cypher Query is:")

query_string = nl2kg_query_engine.generate_query(question)

display(
    Markdown(
        f"""
```cypher
{query_string}
```
"""
    )
)

%ngql {query_string}

# Answer:

print("The Answer is:")

display(Markdown(f"<b>{response_nl2kg}</b>"))

The Cypher Query is:



```cypher
MATCH (p:`entity`)-[:`relationship`]->(q:`entity`)
WHERE p.`name` == 'San Jose Police Department' AND q.`name` == 'Patrol duties'
RETURN p.`name`, q.`name`
```


INFO:nebula3.logger:Get connection to ('172.29.192.1', 9669)
The Answer is:


<b>The San Jose Police Department has an interest in maximizing the number of officers and sergeants available for Patrol duties in order to ensure public safety and maintain law and order in the community. By having a larger number of officers and sergeants on patrol, the police department can effectively respond to emergencies, prevent and deter crime, and provide a visible presence in the community. This can help to reduce response times, increase police visibility, and enhance the overall effectiveness of law enforcement efforts in San Jose.</b>